<a href="https://colab.research.google.com/github/achiral/mosaic_art/blob/main/gh_mosaic_art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### github clone

In [1]:
!git clone https://github.com/achiral/mosaic_art.git

Cloning into 'mosaic_art'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 24 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


### preparation

In [8]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# make dataset directories
!mkdir ./dataset
!mkdir /content/output

# material: blocks.zip
# !cp "./gdrive/MyDrive/dataset/blocks.zip" "./dataset"

# target image (*.png, *.jpg)
# !cp "./gdrive/MyDrive/dataset/target.png" "./dataset"

# unzip material dataset
!unzip ./dataset/blocks.zip -d ./dataset
!unzip ./mosaic_art/blocks.zip -d ./dataset

# count data
!find dataset/* -type f | wc -l

unzip:  cannot find or open ./dataset/blocks.zip, ./dataset/blocks.zip.zip or ./dataset/blocks.zip.ZIP.
Archive:  ./mosaic_art/blocks.zip
   creating: ./dataset/blocks/
  inflating: ./dataset/__MACOSX/._blocks  
  inflating: ./dataset/blocks/weathered_cut_copper.png  
  inflating: ./dataset/__MACOSX/blocks/._weathered_cut_copper.png  
  inflating: ./dataset/blocks/glass_blue.png  
  inflating: ./dataset/__MACOSX/blocks/._glass_blue.png  
  inflating: ./dataset/blocks/glass_pink.png  
  inflating: ./dataset/__MACOSX/blocks/._glass_pink.png  
  inflating: ./dataset/blocks/azalea_leaves.png  
  inflating: ./dataset/__MACOSX/blocks/._azalea_leaves.png  
  inflating: ./dataset/blocks/coral_yellow_dead.png  
  inflating: ./dataset/__MACOSX/blocks/._coral_yellow_dead.png  
  inflating: ./dataset/blocks/rail_activator.png  
  inflating: ./dataset/__MACOSX/blocks/._rail_activator.png  
  inflating: ./dataset/blocks/kelp_top_bulb.tga  
  inflating: ./dataset/__MACOSX/blocks/._kelp_top_bulb.tga  

### run mosaic art

In [ ]:
# %cd /content

In [3]:
import numpy as np
import cv2
import glob

In [6]:
# IMG_PATH = './images/*.png'          # material image path
IMG_PATH = R"./dataset/blocks/*.png"  # material image path

# INPUT_PATH = R"./dataset/target.png"  # target image path
INPUT_PATH = R"./mosaic_art/target.jpg"  # target image path
MULTIPLY = 10                           # magnify target image

In [ ]:
# files = glob.glob('./images/*.png')
files = glob.glob(IMG_PATH)
size = 50    # resize into 50x50
 
small_img = []
 
for file in files:
    image = cv2.imread(file)
    image = cv2.resize(image, (size, size))
    image = image /255
    small_img.append(image)
 
small_img = np.asarray(small_img)
pic_filename = INPUT_PATH                 # target image path
 
big_img = cv2.imread(pic_filename)
height = (big_img.shape[0] - big_img.shape[0] % size) * 2 * MULTIPLY
width = (big_img.shape[1] - big_img.shape[1] % size) * 2 * MULTIPLY
 
big_img = big_img / 255
big_img = np.asarray(big_img)
big_img = cv2.resize(big_img, (width, height))
 
for i in range(int(height / size)):
    print('%d / %d' %(i + 1, int(height / size)))
 
    for j in range(int(width / size)):
        cut = big_img[size * i: size * (i + 1), size * j: size * (j + 1)]
        losses = [] 
        
        # search marial image (small_img) similar to target image color
        for s_img in small_img:
            loss = cut - s_img
            loss = loss * loss
            loss = np.sum(loss)
            losses.append(loss) 
     
        losses = np.asarray(losses)
        big_img[size * i: size * (i + 1), size * j: size * (j + 1)] = small_img[np.argmin(losses)]
 
big_img = (big_img * 255).astype(np.int64)
cv2.imwrite('./output/out.png', big_img)

### colab2drive

In [9]:
!cp "./output/out.png" "./gdrive/MyDrive/dataset/out.png"
print("----- copied image file to google drive (^o^)/ -----")

----- finished -----


### colab2localPC

In [10]:
from google.colab import files
files.download('./output/out.png')
print(">>>--->>>--->>> now downloading image file >>>--->>>--->>>")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

----- downloaded image file -----
